### Optimisation
Optimise parameter set and do single model run with the resulting parameters. Then illustrate results with basic plots.

In [ ]:
import numpy as np
import pymc as pm

from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm

from aust_covid.constants import set_project_base_path
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from aust_covid.plotting import plot_single_run_outputs
from emutools.tex import DummyTexDoc
from emutools.utils import load_param_info, round_sigfig

project_paths = set_project_base_path("../")
SUPPLEMENT_PATH = project_paths["SUPPLEMENT_PATH"]

In [ ]:
max_iterations = 50

In [ ]:
param_info = load_param_info()
dummy_doc = DummyTexDoc()
param_info['value'].update(get_ifrs(dummy_doc))
parameters = param_info['value'].to_dict()
epi_model = build_model(dummy_doc, param_info['abbreviations'], mobility_ext=True, cross_ref=False)
priors = get_priors(False, param_info['abbreviations'], dummy_doc)
prior_names = [p.name for p in priors]
targets = get_targets(dummy_doc)
bcm = BayesianCompartmentalModel(epi_model, parameters, priors, targets)

In [ ]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *bcm.priors[k].bounds(0.99)) for k, v in parameters.items() if k in bcm.priors}
    variables = epm.use_model(bcm)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False, maxeval=max_iterations)
    map_params = {k: float(v) for k, v in map_params.items()}
print('Best candidate parameters found:')
for i_param, param in enumerate([p for p in map_params if '_dispersion' not in p]):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')
parameters.update(map_params)
epi_model.run(parameters=parameters)

In [ ]:
parameters.update(map_params)
epi_model.run(parameters=parameters)

In [ ]:
plot_single_run_outputs(epi_model, targets)